# This file is to take the .xlsx file created by reach_Curator_py38_v3.py script, and adds in the missing rows 
-  this file --> xlsx_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001_Grant_curate_20250421.xlsx"
-  saves out this file --> xlsx_save_path = rf"{main_path}\Grant_curate\manual_additions"

In [89]:
import pandas as pd
import numpy as np
import os
import shutil


## Modify these three paths as needed

In [90]:
root_path = r'G:\Grant\behavior_data\DLC_net' # change this to your root path where all the analysis folder live, i currently use is G:\Grant\behavior_data\DLC_net
root_folder = 'grant_reach10_swingDoor-christie-2025-04-21' # change this to your root folder for one specfic mouse (this should contain multiple sessions)

session_option = 'A'

### Create new session_option for each session

In [91]:
main_path = rf'{root_path}\{root_folder}'
print(f'main_path: {main_path}')

###### mouse session option A
if session_option == 'A':
    mouse = 'reach10'
    session_date = '20250421'
    session_ID = 'session001'


###### mouse session option B'
if session_option == 'B':
    mouse = 'reach10'
    session_date = '20250424'
    session_ID = 'session001'


    ###### mouse session option B'
if session_option == 'C':
    print("⚠️Info for sesion C does not exist yet⚠️")
    mouse = 'reach10'
    session_date = ' '
    session_ID = ' '

main_path: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21


In [93]:
if session_option == 'A':
    xlsx_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001_Grant_curate_20250421.xlsx"
    txt_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\20250421\christielab\session001\20250421_christielab_session001_events_shifted.txt"
    xlsx_save_path = rf"{main_path}\Grant_curate\manual_additions\{mouse}_{session_date}_{session_ID}_manual_additions.xlsx"


if session_option == 'B':
    # Set path_frontCam and path_sideCam for session B manually
    xlsx_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250424_christielab_session001_Grant_curate_20250421.xlsx"
    txt_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\20250424\christielab\session001\20250424_christielab_session001_events_shifted.txt"
    xlsx_save_path = rf"{main_path}\Grant_curate\manual_additions\{mouse}_{session_date}_{session_ID}_manual_additions.xlsx"


# Check if files exist
if not os.path.exists(xlsx_file):
    print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
if not os.path.exists(txt_file):
    print(f"⚠️Warning: {txt_file} does not exist⚠️")
if not os.path.exists(xlsx_save_path):
    print(f"⚠️Warning: {xlsx_save_path} does not exist⚠️")
    os.makedirs(xlsx_save_path)
    print(f"✅ Created directory: {xlsx_save_path}")


In [94]:
# ---- Step 1: Make a backup in curator_backup folder
original_dir = os.path.dirname(xlsx_file)
backup_dir = os.path.join(original_dir, 'curator_direct_backup')
os.makedirs(backup_dir, exist_ok=True)

backup_file_path = os.path.join(backup_dir, os.path.basename(xlsx_file))
shutil.copy2(xlsx_file, backup_file_path)
print(f"✅ Backup saved to: {backup_file_path}")

# ---- (your parsing & merging code goes here)
# ... your code to build `combined` DataFrame ...



✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\curator_direct_backup\20250421_christielab_session001_Grant_curate_20250421.xlsx


### This is the cell that takes the .xlsx file and the .txt file and creates a new .xlsx file with the manual additions
- makes a new folder called manual_additions if it does not exist
- reads the .xlsx file into a pandas dataframe
- reads the .txt file into a pandas dataframe
- creates a new dataframe with the manual additions
- saves the new dataframe to a new .xlsx file



In [84]:
# Load existing curated xlsx
df_orig = pd.read_excel(xlsx_file)
df_orig.head()

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Grant\\behavior_data\\DLC_net\\grant_reach10_swingDoor-christie-2025-04-21\\Grant_curate\\20250421_christielab_session001_Grant_curate_20250421.xlsx'

In [85]:

# Normalize column names just in case
df_orig.columns = df_orig.columns.str.strip()

# Load and parse txt as done earlier
events = []
current_event = {}
with open(txt_file, 'r') as f:
    for line in f:
        if line.strip():
            key, value = line.strip().split('\t')
            value = int(value)
            if key == 'T6000_played':
                if current_event:
                    events.append(current_event)
                current_event = {'T6000': value}
            elif key == 'pellet_detected':
                current_event['pellet_detected'] = value
            elif key == 'pellet_delivery':
                current_event['pellet_delivery'] = value
            elif key == 'T5000_played':
                current_event['T5000'] = value
if current_event:
    events.append(current_event)

events_df = pd.DataFrame(events)

# Find which T6000s are missing from original file
existing_T6000s = df_orig['T6000'].values
new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()

# Fill in missing columns to match structure
new_rows['reachInit'] = np.nan
new_rows['reachMax'] = np.nan
new_rows['reachEnd'] = np.nan
new_rows['stim'] = 0
new_rows['behaviors'] = np.nan

# # Reorder columns to match original file
# new_rows = new_rows[['reachinit', 'reachmax', 'reachend', 'stim',
#                      'pellet_delivery', 'pellet_detected', 't6000', 't5000', 'behaviors']]

# Reorder columns to match original file
new_rows = new_rows[['T6000', 'T5000', 'reachInit', 'reachMax',
                     'reachEnd', 'stim','behaviors' ,'pellet_delivery', 'pellet_detected']]

# Combine and sort by T6000
combined = pd.concat([df_orig, new_rows], ignore_index=True)
combined = combined.sort_values(by='T6000').reset_index(drop=True)

# Force final column order
combined = combined[['T6000', 'T5000', 'reachInit', 'reachMax',
                     'reachEnd', 'stim', 'behaviors',
                     'pellet_delivery', 'pellet_detected']]


# ---- Step 2: Save output as the same original file (overwrite it)
xlsx_save_path = xlsx_file
combined.to_excel(xlsx_save_path, index=False)
print(f"✅ Final merged Excel saved to: {xlsx_save_path}")


print(f"✅ Final merged CSV saved: {xlsx_save_path}")


✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001_Grant_curate_20250421.xlsx
✅ Final merged CSV saved: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001_Grant_curate_20250421.xlsx


In [ ]:
# Load existing curated xlsx
df_modified = pd.read_excel(xlsx_save_path)
df_modified.head(10)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1768,2205.0,NaN,NaN,NaN,0,NaN,2203.0,1823.0
1,1774,2063.0,NaN,NaN,NaN,0,NaN,NaN,NaN
2,1774,2063.0,2077.0,2082.0,2087.0,0,none,NaN,NaN
3,4122,4373.0,NaN,NaN,NaN,0,NaN,NaN,NaN
4,4255,4388.0,NaN,NaN,NaN,0,NaN,4385.0,4308.0


In [88]:
df_orig.head(10)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1774,2063.0,NaN,NaN,NaN,0,NaN,NaN,NaN
1,1774,2063.0,2077.0,2082.0,2087.0,0,none,NaN,NaN
2,4122,4373.0,NaN,NaN,NaN,0,NaN,NaN,NaN
3,7718,8151.0,NaN,NaN,NaN,0,NaN,NaN,NaN
4,10210,10576.0,NaN,NaN,NaN,0,NaN,NaN,NaN
5,13081,13440.0,13456.0,13476.0,13522.0,0,stalled,13438.0,13137.0
6,17190,17407.0,NaN,NaN,NaN,0,NaN,NaN,NaN
7,19496,20128.0,20159.0,20173.0,20194.0,0,grabbed,20126.0,19555.0
8,22224,22704.0,22722.0,22743.0,22750.0,0,grabbed,22701.0,22287.0
9,25965,26249.0,NaN,NaN,NaN,0,NaN,NaN,NaN
